Вариант I

Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)

Логин тестового ящика: study.ai_172@mail.ru

Пароль тестового ящика: NextPassword172#

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common import exceptions
from pymongo import MongoClient
from  pprint import pprint

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client['my_database']
mails = db.mails

In [3]:
options = Options()
options.add_argument('start_maximized')
s = Service('chromedriver.exe')
driver = webdriver.Chrome(service = s, options = options)
driver.implicitly_wait(10)

In [4]:
def mail_parser(url):
    driver.get(url)
    elem = driver.find_element(By.XPATH, "//input[@name = 'username']")
    elem.send_keys('study.ai_172@mail.ru')
    button = driver.find_element(By.XPATH, "//button[@data-test-id = 'next-button']")
    button.submit()
    #time.sleep(5)
    
    elem = driver.find_element(By.XPATH, "//input[@name = 'password']")
    elem.send_keys('NextPassword172#')
    elem.send_keys(Keys.ENTER)
    
    url_list = []
    previous = []
    while True:
        articles = driver.find_elements(By.XPATH, "//a[contains(@class, 'js-letter-list-item')]")       
        if articles != previous:
            for url in articles:
                url = url.get_attribute('href')
                url_list.append(url)
            actions = ActionChains(driver)
            actions.move_to_element(articles[-1])
            actions.perform()
            #time.sleep(1)
            previous = articles
        else:
            print('Скроллинг окончен')
            break
            
    url_list = list(set(url_list))
    
    url_list.remove(None)
    

    for url in(url_list):
        info = {}
        try:
            page = driver.get(url)
            from_ = driver.find_element(By.CLASS_NAME, 'letter-contact').text
            date_ = driver.find_element(By.CLASS_NAME, 'letter__date').text
            theme_ = driver.find_element(By.CLASS_NAME, 'thread-subject').text
            only_text = driver.find_element(By.CLASS_NAME, 'letter-body').text
            info['from_'] = from_
            info['date_'] = date_
            info['theme'] = theme_
            info['text'] = only_text
            mails.insert_one(info)
        except exceptions.NoSuchElementException:
            print('Элемент не найден')
            break

In [5]:
url ='https://account.mail.ru/login'

mail_parser(url)

Скроллинг окончен
Элемент не найден


In [6]:
for doc in mails.find({'from_':'Discord'}):
    pprint(doc)

{'_id': ObjectId('625b14ae72658711ed535041'),
 'date_': '30 марта, 20:36',
 'from_': 'Discord',
 'text': 'Включить уведомления\n'
         'Загрузите Discord на свой телефон и получайте push-уведомления о '
         'событиях в Discord, таких как сообщения, упоминания, запросы на '
         'добавление в друзья и мероприятия.\n'
         'Скачать приложение\n'
         'Привет, Orleon (Иван Иванов)!\n'
         'Пока вы отсутствовали, в Discord вовсю шла движуха! Вот основные '
         'моменты:\n'
         '#music_info (GeekSchool Chat_bots)\n'
         '#general (GeekSchool Chat_bots)\n'
         '  GeekSchool_bot\n'
         '  Илья Акчурин\n'
         '!gif\n'
         '  Илья Акчурин\n'
         '!gif love\n'
         '  Илья Акчурин\n'
         '!gif cucumber\n'
         '  Илья Акчурин\n'
         '!gif\n'
         '#gifs (GeekSchool Chat_bots)\n'
         '  GeekSchool_bot\n'
         'https://giphy.com/gifs/dog-i-love-you-chippythedog-XFpaPUu44PqnsmO9lc\n'
         '  GeekSch

In [7]:
#mails.delete_many({})